In [83]:
import gradio as gr
import ollama 

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

# AI Tutor System Prompt
system_prompt = """
You are **Masterly AI Tutor**, a 24/7 instant doubt-clearing assistant specialized in helping students prepare for master’s entrance exams.
Your task is to provide **accurate, clear, and concise answers** to user queries, focusing on the master’s exam syllabus, concepts, problem-solving
techniques, and past exam patterns.

### **Instructions for Response Generation:**
- **Accuracy & Clarity:** Deliver precise, well-structured explanations in **Markdown format**.
- **Context Awareness:** Understand the query’s domain (math, reasoning, verbal, subject-specific) and ask clarifying questions if needed.
- **Exam-Specific Knowledge:** Incorporate insights from previous exams, and suggest shortcuts or strategies for efficient problem-solving.
- **Engagement & Support:** Maintain a friendly, motivating tone. Offer further study resources or practice exercises as applicable.
- **Subject Coverage:** Address all areas of master’s exams (Quantitative Aptitude, Logical Reasoning, Verbal Ability, General Awareness, and Subject-Specific Topics).
- **Follow-Up & Reinforcement:** Suggest additional practice or deeper breakdowns for complex topics. Provide insights on weak areas based on common queries.
- **Adaptive Learning:** Tailor explanations to the user’s difficulty level. Simplify complex topics when necessary.
- **Multi-Language Support:** Offer explanations in the user’s preferred language, leveraging Google Translate when needed.
"""

def respond_to_user(user_input , subject , image , chat_history):
    if not subject:
        return chat_history + [("Please select a subject before asking a question.", "")]
        
    messages = [
        {"role":"system","content":system_prompt},
    ]
    for user , ai in chat_history:
        messages.append({"role":"user","content":user})
        messages.append({"role":"assistant","content":ai})

    if user_input:
        messages.append({"role": "user", "content": user_input})

    if image:
        messages.append({"role": "user", "content": {"type": "image", "image": image}})
        
    try:
        response = ollama.chat(model="llama3.2", messages=messages)
        ai_reply = response.get("message", {}).get("content", "No response received.")
        return chat_history + [(user_input, ai_reply)]
    
    except Exception as e:
        return chat_history + [(user_input, f" Error: {str(e)}")]

with gr.Blocks(css="""
:root {
    --background-fill-primary: #121212 !important;
    --text-primary: #E0E0E0 !important;
}

/* Headings */
.gradio-container h1, .gradio-container h2 {
    color: #FFFFFF !important;
    font-weight: 900 !important;
    text-transform: uppercase;
    text-shadow: 2px 2px 8px rgba(255, 255, 255, 0.2);
    letter-spacing: 1.2px;
}

/* Subheadings & Bold Text */
.gradio-container .markdown strong {
    color: #FFD700 !important;
    font-weight: 800 !important;
}
""") as tutor_ui:
    gr.Markdown("## 🎓 Master's AI Tutor - 24/7 Instant Doubt Solver")
    gr.Markdown("Ask doubts from **Quant, Verbal, Logical Reasoning, General Awareness** & Subject-Specific topics!")


    with gr.Row():
        subject_dropdown = gr.Dropdown(["Quantitative Aptitude", "Logical Reasoning", "Verbal Ability", "General Awareness", "Subject-Specific"], label="📚 Select Subject")
        image_upload = gr.Image(label="📷 Upload an Image (Optional)", type="pil")

    chat_history = gr.State([])
    chatbot = gr.Chatbot(label="📖 Tutor's Response")
    user_input = gr.Textbox(label="💡 Ask a question...")

    with gr.Row():
        ask_btn = gr.Button("🚀 Ask AI")
        clear_btn = gr.Button("🗑️ Clear Chat")

    ask_btn.click(respond_to_user, inputs=[user_input, subject_dropdown, image_upload, chat_history], outputs=chatbot)
    clear_btn.click(lambda: [], outputs=chatbot)


tutor_ui.launch( share = True)

C:\Users\ASUS\anaconda3\Lib\site-packages\gradio\components\chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7903
* Running on public URL: https://5243ff82125e7ddbab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
